# Image2Graph functions

In [ ]:
import numpy as np

In [2]:
def random_coord(dims=[28,28], N=400):
    coord=[ [i,j] for i in range(dims[0]) for j in range(dims[1]) ]
    perm=np.random.permutation(coord)
    return perm[:N]

In [3]:
def compute_f(I, index):
    f=[]
    for i,j in index:
        f.append(I[:,i,j])
    return np.array(f).T

In [36]:
def top_k(row, k=8):
    r=np.array(row)
    index=r.argsort()[-k]
    cutoff=r[index]
    return [0 if r<cutoff else r  for r in row]

def compute_A( k, index):
    N=len(index)
    A=[]
    for i in range(N):
        row = [ np.linalg.norm(np.array(index[i]) - np.array(index[j]) ) for j in range(N)]
        A.append(top_k(row, k))
    A=np.array(A)
    return A/np.max(A)

In [37]:
def image2graph_setup( N, k, dims=[28,28]):
    index=random_coord(dims, N)
    A=compute_A(k, index)
    return index, A


In [44]:
index, A= image2graph_setup(N=5, k=3)
f=compute_f(Img, index)

# Tensorflow layers

In [ ]:
def Identity(n):
    with tf.name_scope("Identity"):
        I=tf.eye(n)
        return I

def bias_var(shape, name='bias'):
    with tf.name_scope(name):
        b = tf.Variable(tf.constant(0.1, shape=shape))
        return b   
    
    
def initH(A, name='H'):
    with tf.name_scope(name):
        b1=bias_var([1], name='a1')
        b2=bias_var([1], name='a2')
        N=int(A.get_shape()[2])
        x= tf.add( tf.multiply( b1 , Identity(N) ) , b2*A )
        return x


In [ ]:
def GraphLayer(f_in, A, F, name='GraphLayer'):
    with tf.name_scope(name):
        # The number of input features is C and output features is F
        C=int(f_in.get_shape()[2])
        output_layer=[]
        for i in range(F):
            V_f=tf.add_n([tf.matmul(initH(A), tf.expand_dims(f_in[:,:,c], axis=2)) for c in range(C)]) + bias_var([1])
            output_layer.append(V_f)    
        return tf.squeeze(tf.stack(output_layer, axis=2), [3])

In [ ]:
def fully_commected():
    f_flat=tf.reshape(f_out, shape=[-1,49*7])
    W_fc=weight_var([49*7, 5])
    b_fc=bias_var([5])
    x_fc=tf.matmul(f_flat, W_fc) + b_fc

In [ ]:
#Input is function f and adjacency matrix A
def graphCNN(f,A):
    I=Identity(tf.shape(f)[0])
    f1=GraphLayer(f, A, 8)
    f2=GraphLayer(f1, A, 16)
    f3=GraphLayer(f2, A, 7)
    return f3

In [ ]:
def fully_commected(x_in, M, name='FullyConnected'):
    with tf.name_scope(name):
        N= int(x_in.get_shape()[1])
        W_fc=weight_var([N, M])
        b_fc=bias_var([M])
        x_out=tf.matmul(f_flat, W_fc) + b_fc
        return x_out

## Tensorflow Models

In [ ]:
class model:
    def __init__(self, N, N_class):
        self.f=tf.placeholder(tf.float32, [None, N,1])
        self.A=tf.placeholder(tf.float32, [None, N, N])
        self.Labels=tf.placeholder(tf.float32, [None, N_class])
        self.current=self.f
        
    def add_conv_layer(self, F):
        self.current= layers.GraphLayer(self.current, self.A, F, name)
        return None
    
    def add_fc_layer(self, N, dropout=None, name='FClayer'):
        f_flat=layers.flatten(self.current)
        if dropout not None:
            f_flat=tf.nn.dropout(f_flat, dropout)
        self.current=layers.fully_connected(f_flat, N)
        return None
    
    def training(self, LearningRate):
        loss=layers.cross_entropy_loss(self.current, self.Labels)
        train_step = tf.train.AdamOptimizer(LearningRate).minimize(loss)
        return training_step

# Data Loading Class

In [118]:
from utils import *
import numpy as np 
import tensorflow as tf
import pickle

class DataProcessor:
	def __init__(self, nodes=25, batchsize=25):
		self.tr_f, self.tr_A, self.tr_l, self.te_f, self.te_A, self.te_l = self._load_dataset(nodes)
		self.N = nodes
		self.BatchSize = batchsize
		self.DataPoints = len(self.tr_f)
        
	def _load_dataset(self, nodes):
		filehandler = open('MNIST_' + str(nodes) + '.pkl', 'rb')
		tr, te = pickle.load(filehandler)
		return np.expand_dims(tr[0], axis = 2 ), tr[1], tr[2], np.expand_dims(te[0] , axis = 2), te[1], te[2]

	def _get_index(self, k, N):
		a = [i for i in range(N)]
		a = np.random.permutation(a)
		return a[:k]
    
	def next_train(self):
		index = self._get_index( self.BatchSize, len(self.tr_f) )
        
		f = self.tr_f[index, :, :]
		A = self.tr_A[index, :, :]
		l = self.tr_l[index, :]
        
		return f, A, l


	def validation(self):
		return self.te_f, self.te_A, self.te_l
        

In [119]:
mnist = DataProcessor(nodes=50)

In [120]:
a,b, c = mnist.next_train()

In [121]:
a.shape

(25, 50, 1)

In [83]:
mnist.train

<BatchDataset shapes: ((?, 784), (?, 10)), types: (tf.float32, tf.float64)>

array(<tf.Tensor 'IteratorGetNext_8:0' shape=(?, 784) dtype=float32>,
      dtype=object)